In [1]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

ko_text = "이것은 m2m모델로 만든 다국어 번역기 입니다."
chinese_text = "生活就像一盒巧克力。"

model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

# 한국어를 영어로
tokenizer.src_lang = "ko"
encoded_hi = tokenizer(ko_text, return_tensors="pt")
generated_tokens = model.generate(**encoded_hi, forced_bos_token_id=tokenizer.get_lang_id("en"))
result1 = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
print(result1)

# translate Chinese to English
tokenizer.src_lang = "zh"
encoded_zh = tokenizer(chinese_text, return_tensors="pt")
generated_tokens = model.generate(**encoded_zh, forced_bos_token_id=tokenizer.get_lang_id("ko"))
result2 = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
print(result2)


['This is a multi-language translator made with the m2m model.']
['인생은 초콜릿 상자와 같다.']


In [10]:
import torch
import gradio as gr
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

# -----------------------------
# 1. 모델 & 토크나이저 로드 (전역에서 1번만)
# -----------------------------
MODEL_NAME = "facebook/m2m100_418M"

tokenizer = M2M100Tokenizer.from_pretrained(MODEL_NAME)
model = M2M100ForConditionalGeneration.from_pretrained(MODEL_NAME)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# UI에서 보여줄 언어 이름 ↔ M2M100 코드 매핑
LANG_MAP = {
    "한국어 (ko)": "ko",
    "영어 (en)": "en",
    "중국어 (zh)": "zh",        # 간체 중국어
    "일본어 (ja)" : "ja",
    "스페인어 (es)" : "es"
}

# -----------------------------
# 2. 번역 함수
# -----------------------------
def translate(text, src_lang_label, tgt_lang_label):
    if not text.strip():
        return "텍스트를 입력하세요."

    src_lang = LANG_MAP[src_lang_label]
    tgt_lang = LANG_MAP[tgt_lang_label]

    # M2M100에 사용할 원문 언어 설정
    tokenizer.src_lang = src_lang

    # 토크나이즈
    encoded = tokenizer(text, return_tensors="pt")
    encoded = {k: v.to(device) for k, v in encoded.items()}

    # target 언어의 BOS 토큰 ID
    forced_bos_token_id = tokenizer.get_lang_id(tgt_lang)

    # 번역 생성
    generated_tokens = model.generate(
        **encoded,
        forced_bos_token_id=forced_bos_token_id,
        max_length=256
    )

    result = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    return result

# -----------------------------
# 3. Gradio UI 정의
# -----------------------------
with gr.Blocks(title="M2M100 다국어 번역 데모") as demo:
    gr.Markdown(
        """
        # 🌐 M2M100 다국어 번역기 데모  
        `facebook/m2m100_418M` 모델을 사용한 간단한 번역 서비스입니다.  
        - 예시 1: 한국어 → 영어  
        - 예시 2: 중국어 → 한국어  
        """
    )

    with gr.Row():
        with gr.Column():
            src_lang = gr.Dropdown(
                label="원문 언어",
                choices=list(LANG_MAP.keys()),
                value="한국어 (ko)"
            )
            
            src_text = gr.Textbox(
                label="입력 텍스트",
                lines=5,
                placeholder="번역할 문장을 입력하세요."
            )
            
        

        with gr.Column():
            tgt_lang = gr.Dropdown(
                label="번역 언어",
                choices=list(LANG_MAP.keys()),
                value="영어 (en)"
            )
            
            
            out_text = gr.Textbox(
                label="번역 결과",
                lines=5
            )
        
    with gr.Row():
        translate_btn = gr.Button("번역하기")

    # 버튼 클릭 시 translate 함수 호출
    translate_btn.click(
        fn=translate,
        inputs=[src_text, src_lang, tgt_lang],
        outputs=out_text
    )

demo.launch()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [9]:
demo.close()

Closing server running on port: 7860


# 한영-영한 번역 후 음성으로 읽어주는 서비스

영어를 읽어주는 모델

In [2]:
# !pip install -q kokoro>=0.9.2 soundfile
# !apt-get -qq -y install espeak-ng > /dev/null 2>&1
from kokoro import KPipeline
from IPython.display import display, Audio
import soundfile as sf
import torch
pipeline = KPipeline(lang_code='a')
text = "The mantra of “complete denuclearization of the Korean Peninsula,” once a strategic objective for Washington and Beijing alike, has vanished from their respective key official strategy documents, as the intensifying US-China rivalry reshapes regional priorities."

generator = pipeline(text, voice='af_heart')
for i, (gs, ps, audio) in enumerate(generator):
    print(i, gs, ps)
    display(Audio(data=audio, rate=24000, autoplay=i==0))
    sf.write(f'{i}.wav', audio, 24000)


/home/haram/miniforge3/envs/hug/lib/python3.10/site-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(
/home/haram/miniforge3/envs/hug/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


0 The mantra of “complete denuclearization of the Korean Peninsula,” once a strategic objective for Washington and Beijing alike, has vanished from their respective key official strategy documents, as the intensifying US-China rivalry reshapes regional priorities. ðə mˈæntɹə ʌv “kəmplˈit dᵻnˌuklɪɹᵻzˈAʃən ʌv ðə kəɹˈiən pənˈɪnsᵊlə,” wˈʌns ɐ stɹətˈiʤɪk əbʤˈɛktɪv fɔɹ wˈɔʃɪŋtən ænd bˌAʤˈɪŋ əlˈIk, hæz vˈænɪʃt fɹʌm ðɛɹ ɹəspˈɛktɪv kˈi əfˈɪʃᵊl stɹˈæTəʤi dˈɑkjəmənts, æz ði ɪntˈɛnsᵻfˌIɪŋ juˌɛsʧˈInə ɹˈIvəlɹi ɹiʃˈAps ɹˈiʤᵊnəl pɹIˈɔɹəTiz.


한국어를 읽어주는 모델

In [3]:
# !pip install uroman

In [4]:
from transformers import VitsModel, AutoTokenizer
import torch
from uroman import Uroman
import soundfile as sf

model = VitsModel.from_pretrained("facebook/mms-tts-kor")
tokenizer = AutoTokenizer.from_pretrained("facebook/mms-tts-kor")

text = "이것은 허깅페이스에서 받는 모델로 만든 한국어 읽어주는 프로그램 입니다."
uroman = Uroman()
roman_text = uroman.romanize_string(text)

inputs = tokenizer(roman_text, return_tensors="pt")

with torch.no_grad():
    output = model(**inputs).waveform
    
# 5. shape 정리
waveform = output.squeeze(0).cpu().numpy()   # shape (N,)

# 6. Notebook에서 바로 재생
display(Audio(waveform, rate=model.config.sampling_rate))   
sf.write("korean_voice.wav", waveform, 24000)
    


In [5]:
import torch
import numpy as np
import gradio as gr
import soundfile as sf

from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
from transformers import VitsModel, AutoTokenizer
from kokoro import KPipeline
from uroman import Uroman

# -----------------------------
# 1. 번역 모델 & 토크나이저 로드
# -----------------------------
MODEL_NAME = "facebook/m2m100_418M"

tokenizer = M2M100Tokenizer.from_pretrained(MODEL_NAME)
model = M2M100ForConditionalGeneration.from_pretrained(MODEL_NAME)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# UI에서 보여줄 언어 이름 ↔ M2M100 코드 매핑
LANG_MAP = {
    "한국어 (ko)": "ko",
    "영어 (en)": "en",
    "중국어 (zh)": "zh",
    "일본어 (ja)": "ja",
    "스페인어 (es)": "es"
}

# -----------------------------
# 2. TTS 모델 로드
#    - 영어: Kokoro
#    - 한국어: MMS-TTS (facebook/mms-tts-kor)
# -----------------------------

# 영어 TTS (Kokoro)
pipeline_en = KPipeline(lang_code='a')  # 사용자가 쓰던 그대로 'a'

# 한국어 TTS (MMS)
ko_tts_model = VitsModel.from_pretrained("facebook/mms-tts-kor")
ko_tts_tokenizer = AutoTokenizer.from_pretrained("facebook/mms-tts-kor")
uroman = Uroman()

# -----------------------------
# 3. 번역 함수
# -----------------------------
def translate(text, src_lang_label, tgt_lang_label):
    if not text.strip():
        return "텍스트를 입력하세요."

    src_lang = LANG_MAP[src_lang_label]
    tgt_lang = LANG_MAP[tgt_lang_label]

    # M2M100에 사용할 원문 언어 설정
    tokenizer.src_lang = src_lang

    # 토크나이즈
    encoded = tokenizer(text, return_tensors="pt")
    encoded = {k: v.to(device) for k, v in encoded.items()}

    # target 언어의 BOS 토큰 ID
    forced_bos_token_id = tokenizer.get_lang_id(tgt_lang)

    # 번역 생성
    generated_tokens = model.generate(
        **encoded,
        forced_bos_token_id=forced_bos_token_id,
        max_length=256
    )

    result = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    return result

# -----------------------------
# 4. TTS 함수들
# -----------------------------
def tts_en(text: str):
    """
    영어 문장을 Kokoro로 합성해서 (sr, audio_ndarray) 반환
    """
    # Kokoro는 여러 segment를 generator로 내보냄 → 전부 이어붙이기
    audio_segments = []
    for _, _, audio in pipeline_en(text, voice='af_heart'):
        audio_segments.append(audio)

    if not audio_segments:
        return None, None

    audio = np.concatenate(audio_segments)
    sr = 24000  # Kokoro 기본 예시 샘플레이트
    return sr, audio

def tts_ko(text: str):
    """
    한국어 문장을 MMS-TTS로 합성해서 (sr, audio_ndarray) 반환
    """
    roman_text = uroman.romanize_string(text)
    inputs = ko_tts_tokenizer(roman_text, return_tensors="pt")

    with torch.no_grad():
        output = ko_tts_model(**inputs).waveform

    waveform = output.squeeze(0).cpu().numpy()
    sr = ko_tts_model.config.sampling_rate  # 보통 24000
    return sr, waveform

# -----------------------------
# 5. 번역 + TTS 통합 래퍼
# -----------------------------
def translate_and_tts(text, src_lang_label, tgt_lang_label):
    translated = translate(text, src_lang_label, tgt_lang_label)
    tgt_lang = LANG_MAP[tgt_lang_label]

    # 기본값: 오디오 없음
    audio_result = None

    # 목표 언어에 따라 TTS 선택
    if tgt_lang == "en":
        sr, audio = tts_en(translated)
        if sr is not None:
            audio_result = (sr, audio)
    elif tgt_lang == "ko":
        sr, audio = tts_ko(translated)
        if sr is not None:
            audio_result = (sr, audio)

    # Gradio에서 (텍스트, 오디오) 둘 다 반환
    return translated, audio_result

# -----------------------------
# 6. Gradio UI 정의
# -----------------------------
with gr.Blocks(title="M2M100 번역 + TTS 데모") as demo:
    gr.Markdown(
        """
        # 🌐 M2M100 번역 + 음성 합성 데모  
        `facebook/m2m100_418M`으로 번역하고,  
        - **한국어 결과**는 `facebook/mms-tts-kor`로  
        - **영어 결과**는 `Kokoro`로  
        읽어주는 서비스입니다.  
        
        ✅ 주 사용 예시  
        - 한국어 → 영어 (영어 음성 재생)  
        - 영어 → 한국어 (한국어 음성 재생)  
        """
    )

    with gr.Row():
        with gr.Column():
            src_lang = gr.Dropdown(
                label="원문 언어",
                choices=list(LANG_MAP.keys()),
                value="한국어 (ko)"
            )
            
            src_text = gr.Textbox(
                label="입력 텍스트",
                lines=5,
                placeholder="번역할 문장을 입력하세요."
            )

        with gr.Column():
            tgt_lang = gr.Dropdown(
                label="번역 언어",
                choices=list(LANG_MAP.keys()),
                value="영어 (en)"
            )
            
            out_text = gr.Textbox(
                label="번역 결과",
                lines=5
            )

            
    with gr.Row():
        out_audio = gr.Audio(
                label="번역 결과 음성",
                autoplay=True
            )
    with gr.Row():
        translate_btn = gr.Button("번역 + 음성 재생")

    # 버튼 클릭 시 번역 + TTS 함수 호출
    translate_btn.click(
        fn=translate_and_tts,
        inputs=[src_text, src_lang, tgt_lang],
        outputs=[out_text, out_audio]
    )

demo.launch()


/home/haram/miniforge3/envs/hug/lib/python3.10/site-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(
/home/haram/miniforge3/envs/hug/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


/home/haram/miniforge3/envs/hug/lib/python3.10/site-packages/gradio/processing_utils.py:688: UserWarning: Trying to convert audio automatically from float32 to 16-bit int format.
  warnings.warn(warning.format(data.dtype))


In [24]:
demo.close()

Closing server running on port: 7861
